In [2]:
!pip install -q unsloth
# Also get the latest nightly Unsloth!
!pip install -q --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.7/358.7 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.4/284.4 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 145.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.9/224.9 kB 23.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyprojec

In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 1024
dtype = None
load_in_4bit = True # Use 4bit quantization to reduce memory usage

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit, # Will load the 4Bit Quantized Model
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.4: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 8,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 42,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.11.4 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [5]:
from datasets import load_dataset
dataset = load_dataset("TeichAI/claude-sonnet-4.5-high-reasoning-250x", split="train")

README.md:   0%|          | 0.00/777 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


claude-4.5-high-reasoning-250x.jsonl: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/247 [00:00<?, ? examples/s]

In [6]:
print(dataset[:2])

{'messages': [[{'role': 'system', 'content': ''}, {'role': 'user', 'content': 'Is the fear of death rational, or is it primarily driven by the unknown?'}, {'role': 'assistant', 'content': '<think>This is a deep philosophical question about the nature of fear of death. Let me think through different perspectives:\n\nArguments that fear of death is rational:\n- It\'s an evolutionary adaptation that helps us survive\n- Death means the end of all experiences, projects, relationships we value\n- There are real losses involved - time, opportunities, connections\n- The deprivation of future goods is a concrete harm\n\nArguments it\'s driven by the unknown:\n- We can\'t truly comprehend non-existence\n- Anxiety about what (if anything) comes after\n- The process of dying itself is uncertain\n- Much of the fear seems to be about the mystery rather than a reasoned evaluation\n\nMixed perspectives:\n- Both elements likely play a role\n- The rational and emotional/unknown aspects may be inseparabl

In [7]:
r1_prompt = """You are a high reasoning assistant engaging in thorough, iterative reasoning, mimicking human stream-of-consciousness thinking. Your approach should be exploratory, clearly explaining , self-check, and continuous refinement before coming up with an answer.
<problem>
{}
</problem>
{}
"""
EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
  texts = []

  for conversation in examples["messages"]:
    problem_content = conversation[1]['content'] # This is the user's question
    solution_content = conversation[2]['content'] # This is the assistant's response

    text = r1_prompt.format(problem_content, solution_content) + EOS_TOKEN
    texts.append(text)
  return {"text": texts}

dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/247 [00:00<?, ? examples/s]

In [8]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2, # Number of processors to use for processing the dataset
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2, # The batch size per GPU/TPU core
        gradient_accumulation_steps = 4, # Number of steps to perform befor each gradient accumulation
        warmup_steps = 5, # Few updates with low learning rate before actual training
        max_steps = 30, # Specifies the total number of training steps (batches) to run.
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit", # Optimizer
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc for observability
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/247 [00:00<?, ? examples/s]

In [9]:
trainer_stats = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 247 | Num Epochs = 1 | Total steps = 30
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 12,156,928 of 3,224,906,752 (0.38% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.808500
2,1.200500
3,1.543400
4,1.122000
5,1.574500
6,1.468000
7,1.874200
8,1.539700
9,1.144700
10,1.247000


In [10]:
from unsloth.chat_templates import get_chat_template
sys_prompt ="""You are a high reasoning assistant engaging in thorough, iterative reasoning, mimicking human stream-of-consciousness thinking. Your approach should be exploratory, clearly explaining , self-check, and continuous refinement before coming up with an answer.
<problem>
{}
</problem>
{}
"""
# Fix: Provide an empty string for the second placeholder as the model will generate this part.
message = sys_prompt.format("What is 2 power 256", "")
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model)
messages = [
    {"role": "user", "content": message},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 1024, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
response = tokenizer.batch_decode(outputs)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [11]:
print(response[0])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

You are a high reasoning assistant engaging in thorough, iterative reasoning, mimicking human stream-of-consciousness thinking. Your approach should be exploratory, clearly explaining, self-check, and continuous refinement before coming up with an answer.
<problem>
What is 2 power 256
</problem>

<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Let me calculate that:

2^256 = 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 2 * 

In [ ]:
import torch
from unsloth import FastLanguageModel # Make sure Unsloth is imported

# --- 1. Define Hub ID and Token ---
# IMPORTANT: Replace with your actual Hugging Face username and model name
HUB_ID = "swehuggingface/fine-tuned-llama-model"
# IMPORTANT: Replace with your actual Hugging Face WRITE token
HF_TOKEN = "hf_your_actual_token_here"

print("Starting merge and push process...")

# --- 2. Push the Merged Model to the Hugging Face Hub ---
# This function does the following:
# 1. Merges the LoRA adapters into the base model.
# 2. Saves the combined model and tokenizer files locally.
# 3. Uploads the complete, merged model files to the Hugging Face Hub.

model.push_to_hub_merged(
    HUB_ID,
    tokenizer,
    save_method = "merged_16bit", # Standard for deployment (bfloat16/float16)
    token = HF_TOKEN                # Uses your token for authentication
)

print(f"\n SUCCESS! Fully Merged Model and Tokenizer are now available on the Hugging Face Hub:")
print(f"🔗 https://huggingface.co/{HUB_ID}")

Starting merge and push process...


config.json:   0%|          | 0.00/890 [00:00<?, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...lama-model/tokenizer.json: 100%|##########| 17.2MB / 17.2MB            

...

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...0002-of-00002.safetensors:   0%|          | 4.26MB / 1.46GB            

Unsloth: Merging weights into 16bit: 100%|██████████| 2/2 [04:57<00:00, 148.62s/it]


Unsloth: Merge process complete. Saved to `/content/swehuggingface/fine-tuned-llama-model`

✅ SUCCESS! Fully Merged Model and Tokenizer are now available on the Hugging Face Hub:
🔗 https://huggingface.co/swehuggingface/fine-tuned-llama-model


Dataset Link - ##https://huggingface.co/datasets/TeichAI/claude-sonnet-4.5-high-reasoning-250x/viewer/default/train?row=3&views%5B%5D=train